In [1]:
import pandas as pd

quota = pd.read_html('quota.html', match= "Quota")

In [2]:
print(len(quota))

1


In [3]:
df = quota[0]
df.head()

,0,1,2,3,4
0,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:
1,Department,Statu,Quota,Current,NaN
2,MANAGEMENT INFORMATION SYSTEMS,UNDERGRADUATE,Unlimited,71,NaN
3,MANAGEMENT INFORMATION SYSTEMS,OZELLISANS,2,1,NaN


In [4]:
df.columns = ['Department', 'Status', "Quota", "Current","a"]
df.head()

,Department,Status,Quota,Current,a
0,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:
1,Department,Statu,Quota,Current,NaN
2,MANAGEMENT INFORMATION SYSTEMS,UNDERGRADUATE,Unlimited,71,NaN
3,MANAGEMENT INFORMATION SYSTEMS,OZELLISANS,2,1,NaN


In [5]:
df.drop("a",axis=1, inplace=True)
df

,Department,Status,Quota,Current
0,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:
1,Department,Statu,Quota,Current
2,MANAGEMENT INFORMATION SYSTEMS,UNDERGRADUATE,Unlimited,71
3,MANAGEMENT INFORMATION SYSTEMS,OZELLISANS,2,1


In [6]:
df.drop(labels=range(0,2), inplace=True)
df = df.reset_index(drop=True)
df

,Department,Status,Quota,Current
0,MANAGEMENT INFORMATION SYSTEMS,UNDERGRADUATE,Unlimited,71
1,MANAGEMENT INFORMATION SYSTEMS,OZELLISANS,2,1


In [7]:
# import requests
# from bs4 import BeautifulSoup
 
 
# url = 'https://registration.boun.edu.tr/scripts/sch.asp?donem=2022/2023-2&kisaadi=MIS&bolum=MANAGEMENT+INFORMATION+SYSTEMS'
# reqs = requests.get(url)
# soup = BeautifulSoup(reqs.text, 'html.parser')
 
# urls = []
# for link in soup.find_all('a'):
#     print(link.get('href'))

In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC

driver1 = webdriver.Chrome()
driver1.get("https://registration.boun.edu.tr/buis/General/schedule.aspx?p=semester")

wait = WebDriverWait(driver1, 10)
element1 = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'input[type="submit"]')))
element_id1 = element1.get_attribute('id')
go_button = driver1.find_element_by_id(element_id1)

go_button.click()

driver1.implicitly_wait(3)

link_elements = driver1.find_elements(By.TAG_NAME, 'a')

urls = []

for link in link_elements:
    urls.append(link.get_attribute('href'))

driver1.quit()

print(urls)

AttributeError: 'WebDriver' object has no attribute 'find_element_by_id'

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import numpy as np

driver = webdriver.Chrome()
driver.get("https://registration.boun.edu.tr/scripts/sch.asp?donem=2022/2023-2&kisaadi=HIST&bolum=HISTORY")

info_links = driver.find_elements(By.LINK_TEXT,"Info")

dfs = []

for info_link in info_links:
    info_link.click()
    # Switch to the new window
    
    new_window = driver.window_handles[-1]
    driver.switch_to.window(new_window)
    
    # Find the table element
    table = driver.find_element(By.TAG_NAME, "table")
    
    # Convert the table content to a DataFrame
    df2 = pd.read_html(table.get_attribute('outerHTML'))[0]
    
    strongs = driver.find_elements(By.TAG_NAME, "strong")
    strongs_text = [strong.text for strong in strongs]
    
    strongs_text_repeated = np.repeat(strongs_text, len(df2))
    
    df2["Code"] = strongs_text_repeated[:len(df2)]
    
    dfs.append(df2)
    
   
    # Close the new window
    driver.close()
    
    # Switch back to the main window
    main_window = driver.window_handles[0]
    driver.switch_to.window(main_window)

result = pd.concat(dfs, ignore_index=True)

In [10]:
result

,0,1,2,3,4,Code
0,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,GR 112.01
1,Department,Statu,Quota,Current,NaN,GR 112.01
2,ALL,ALL,Consent Of Instructor,7,NaN,GR 112.01
3,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,GR 212.01
4,Department,Statu,Quota,Current,NaN,GR 212.01
...,...,...,...,...,...,...
441,Department,Statu,Quota,Current,NaN,LAT 112.01
442,ALL,ALL,20,13,NaN,LAT 112.01
443,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,LAT 212.01
444,Department,Statu,Quota,Current,NaN,LAT 212.01


In [11]:
result = result[["Code",0,1,2,3,4]]
result

,Code,0,1,2,3,4
0,GR 112.01,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:
1,GR 112.01,Department,Statu,Quota,Current,NaN
2,GR 112.01,ALL,ALL,Consent Of Instructor,7,NaN
3,GR 212.01,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:
4,GR 212.01,Department,Statu,Quota,Current,NaN
...,...,...,...,...,...,...
441,LAT 112.01,Department,Statu,Quota,Current,NaN
442,LAT 112.01,ALL,ALL,20,13,NaN
443,LAT 212.01,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:
444,LAT 212.01,Department,Statu,Quota,Current,NaN


In [12]:
result.drop(result.columns[[5]], axis=1, inplace=True)
result

C:\Users\Asus\AppData\Local\Temp\ipykernel_19120\1594288655.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result.drop(result.columns[[5]], axis=1, inplace=True)


,Code,0,1,2,3
0,GR 112.01,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:
1,GR 112.01,Department,Statu,Quota,Current
2,GR 112.01,ALL,ALL,Consent Of Instructor,7
3,GR 212.01,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:
4,GR 212.01,Department,Statu,Quota,Current
...,...,...,...,...,...
441,LAT 112.01,Department,Statu,Quota,Current
442,LAT 112.01,ALL,ALL,20,13
443,LAT 212.01,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:
444,LAT 212.01,Department,Statu,Quota,Current


In [13]:
result.columns = ['Code', 'Department', 'Status', 'Quota', 'Current Quota']
result

,Code,Department,Status,Quota,Current Quota
0,GR 112.01,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:
1,GR 112.01,Department,Statu,Quota,Current
2,GR 112.01,ALL,ALL,Consent Of Instructor,7
3,GR 212.01,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:
4,GR 212.01,Department,Statu,Quota,Current
...,...,...,...,...,...
441,LAT 112.01,Department,Statu,Quota,Current
442,LAT 112.01,ALL,ALL,20,13
443,LAT 212.01,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:
444,LAT 212.01,Department,Statu,Quota,Current


In [14]:
words = ["Departmental Quotas:", "Department", "OZELLISANS", "PHD","MASTER","OZELMASTER","OZELPHD"]

result = result[result.Department.isin(words) == False]
result = result[result.Status.isin(words) == False]
result.reset_index(drop=True, inplace=True)
result

,Code,Department,Status,Quota,Current Quota
0,GR 112.01,ALL,ALL,Consent Of Instructor,7
1,GR 212.01,ALL,ALL,Consent Of Instructor,5
2,HIST102.01,ALL,ALL,Consent Of Instructor,0
3,HIST102.01,HISTORY,ALL,Unlimited,85
4,HIST106.01,CIVIL ENGINEERING,UNDERGRADUATE,2,2
...,...,...,...,...,...
235,HIST790.13,ALL,ALL,Consent Of Instructor,0
236,HIST790.14,ALL,ALL,Consent Of Instructor,0
237,HIST790.15,ALL,ALL,Consent Of Instructor,1
238,LAT 112.01,ALL,ALL,20,13


In [15]:
# import openpyxl
# file_name = "QuotaTesting.xlsx"

# result.to_excel(file_name)